In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

base_path = "/content/drive/MyDrive/SmartVision_Project/smartvision_dataset/classification"

train_dir = os.path.join(base_path, "train")
val_dir   = os.path.join(base_path, "val")
test_dir  = os.path.join(base_path, "test")

print("Train exists:", os.path.exists(train_dir))
print("Val exists:", os.path.exists(val_dir))
print("Test exists:", os.path.exists(test_dir))

Train exists: True
Val exists: True
Test exists: True


In [4]:
classes = sorted(os.listdir(train_dir))
num_classes = len(classes)

print("Class names:", classes)
print("Number of classes:", num_classes)

Class names: ['airplane', 'bed', 'bench', 'bicycle', 'bird', 'bottle', 'bowl', 'bus', 'cake', 'car', 'cat', 'chair', 'couch', 'cow', 'cup', 'dog', 'elephant', 'horse', 'motorcycle', 'person', 'pizza', 'potted plant', 'stop sign', 'traffic light', 'train', 'truck']
Number of classes: 26


Create Image Generators (EfficientNet preprocessing)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1821 images belonging to 26 classes.
Found 390 images belonging to 26 classes.
Found 390 images belonging to 26 classes.


Model 3: EfficientNetB0

In [6]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

# Freeze base model first
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │        13,338 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,723,901 (18.02 MB)

 Trainable params: 671,770 (2.56 MB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [7]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 670s 11s/step - accuracy: 0.2737 - loss: 3.1210 - val_accuracy: 0.4077 - val_loss: 2.1000
Epoch 2/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 169s 3s/step - accuracy: 0.6736 - loss: 1.1336 - val_accuracy: 0.4718 - val_loss: 1.9224
Epoch 3/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 186s 3s/step - accuracy: 0.7677 - loss: 0.8299 - val_accuracy: 0.4538 - val_loss: 1.9659
Epoch 4/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.7963 - loss: 0.6852 - val_accuracy: 0.4590 - val_loss: 2.1121
Epoch 5/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 182s 3s/step - accuracy: 0.8271 - loss: 0.6072 - val_accuracy: 0.4897 - val_loss: 2.1133


Fine-Tune EfficientNet

In [8]:
# Unfreeze top layers of EfficientNet
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=0.0001),  # lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │        13,338 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,723,901 (18.02 MB)

 Trainable params: 2,167,930 (8.27 MB)

 Non-trainable params: 2,555,971 (9.75 MB)

In [9]:
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3
)

Epoch 1/3
57/57 ━━━━━━━━━━━━━━━━━━━━ 214s 3s/step - accuracy: 0.7885 - loss: 0.7468 - val_accuracy: 0.4821 - val_loss: 2.2963
Epoch 2/3
57/57 ━━━━━━━━━━━━━━━━━━━━ 194s 3s/step - accuracy: 0.8540 - loss: 0.5302 - val_accuracy: 0.4744 - val_loss: 2.4277
Epoch 3/3
57/57 ━━━━━━━━━━━━━━━━━━━━ 192s 3s/step - accuracy: 0.8684 - loss: 0.4709 - val_accuracy: 0.4821 - val_loss: 2.5190


EfficientNet Final Result

Train accuracy → ~87% ✅

Validation accuracy → ~48% ⚠️

Same validation ceiling as ResNet (~50%)

In [10]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

13/13 ━━━━━━━━━━━━━━━━━━━━ 102s 8s/step - accuracy: 0.4955 - loss: 2.4431
Test Accuracy: 0.47948718070983887


Final EfficientNet Performance

Train accuracy: ~87%

Validation accuracy: ~48%

Test accuracy: 47.95%

In [12]:
model.save('/content/drive/MyDrive/SmartVision_Project/efficientnet_model.h5')
print("EfficientNet model saved!")


EfficientNet model saved!
